<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Soluci%C3%B3n.%20Generador%20de%20Res%C3%BAmenes%20Inteligente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
!pip install langchain -qU  # Instalar la librería principal de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
!pip install langchain-openai -qU
!pip install langchain-groq -qU
!pip install langchain-google-genai -qU
!pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint


from IPython.display import Markdown

In [4]:
llm = ChatOpenAI(model="gpt-4o",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

In [5]:
# Definir las plantillas
template_sistema = """\
Eres un asistente experto en resumir textos largos. Tu tarea es generar dos tipos de resúmenes para el texto que recibas:

1. Resumen breve: Un resumen conciso de 3 a 4 oraciones que capture las ideas principales del texto.
2. Resumen detallado: Un resumen más extenso, de al menos un párrafo, que incluya detalles clave, ejemplos o puntos importantes del texto.

Instrucciones:
- Lee siempre el texto cuidadosamente.
- Identifica las ideas principales y los detalles relevantes.
- Genera primero el resumen breve y luego el resumen detallado.
- Asegúrate de que ambos resúmenes sean claros, coherentes y fieles al contenido original.

Ejemplo de formato de salida:
Resumen breve:   El cambio climático es un desafío global que causa el aumento de temperaturas, el derretimiento de glaciares y la subida del nivel del mar. Sus efectos impactan negativamente en los ecosistemas y las comunidades humanas. Para mitigarlo, es crucial reducir las emisiones y adoptar energías renovables.
Resumen detallado: El cambio climático representa uno de los problemas más urgentes de la actualidad, caracterizado por el aumento de las temperaturas globales, el derretimiento acelerado de los glaciares y el incremento del nivel del mar. Estos fenómenos tienen consecuencias graves, como la pérdida de biodiversidad, la destrucción de ecosistemas y el desplazamiento de comunidades humanas. Para enfrentar este desafío, es fundamental implementar medidas como la reducción de emisiones de gases de efecto invernadero, la transición hacia energías renovables y la adopción de prácticas sostenibles en la industria, la agricultura y otros sectores clave.


"""

template_usuario = """\
Texto a resumir: {mensaje_usuario}

"""

In [6]:
# Crear el ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", template_sistema),
    ("human", template_usuario),
])

In [12]:
texto_input = input("Escribe aquí el texto a resumir")

Escribe aquí el texto a resumirLa sociología es la ciencia social que se encarga del análisis científico de la sociedad humana o población regional. De manera sucinta se puede decir que esta estudia la sociedad humana, a los grupos humanos y las relaciones que forman la sociedad. Ello significa que la sociología analiza las relaciones (de producción, distribución, consumo, solidaridad, división del trabajo, etc.) que se establecen entre esos grupos humanos (la familia, los clubes, las pandillas, las asociaciones, las instituciones, etc.). En general pretende dar cuenta de las interacciones sociales, las instituciones, la producción y las formas de sociedades existentes.  En la sociología se utilizan metodologías de investigación interdisciplinarias para el análisis e interpretación, desde diversas perspectivas teóricas, de las causas y significados que motivan la aparición de diversas tendencias de comportamiento social. Mientras algunos sociólogos realizan investigaciones que pueden a

In [13]:
    # Formatear el prompt con el mensaje del usuario y el idioma
    prompt = chat_prompt_template.format(
        mensaje_usuario=texto_input,

    )

    # Obtener la respuesta del modelo
    respuesta = llm.invoke(prompt)
    display(Markdown(respuesta.content))

Resumen breve: La sociología es la ciencia social que estudia la sociedad humana, analizando las relaciones e interacciones entre grupos e individuos. Utiliza metodologías interdisciplinarias para comprender tendencias de comportamiento social, abarcando desde la microsociología hasta la macrosociología. Los sociólogos investigan temas como estratificación social, religión, economía y más, contribuyendo tanto a la política social como al conocimiento teórico.

Resumen detallado: La sociología se dedica al análisis científico de la sociedad humana, enfocándose en las relaciones e interacciones que forman la estructura social. Esta disciplina examina las diversas formas de interacción social, las instituciones y las sociedades existentes, aplicando metodologías de investigación interdisciplinarias. Desde la microsociología, que estudia interacciones y organizaciones, hasta la macrosociología, que se centra en sistemas y estructuras sociales, los sociólogos investigan temas como la estratificación social, la movilidad, la religión y la desviación social. La sociología ha ampliado su enfoque a áreas como la economía, la salud, la educación y la criminología. Sus orígenes se asocian a pensadores como Alexis de Tocqueville, Karl Marx y Max Weber, mientras que en el siglo XX destacan figuras como Talcott Parsons y Michel Foucault. En la actualidad, sociólogos como Anthony Giddens y Zygmunt Bauman continúan influyendo en la disciplina.